In [5]:
# Trail.1.ipynb (Final Working Version)
from DSSATTools import (
    crop,
    WeatherStation,
    SoilProfile,
    filex,
    DSSAT
)
import pandas as pd
from datetime import datetime, timedelta
import os
import shutil

# --- DEFINE FILE PATHS AND DATES ---
WEATHER_FILE_PATH = r"D:\Final-Year-Project\project-trail\defaults\Weather\ALCL5601.WTH" 
SOIL_FILE_PATH = "MY_GENERATED_SOIL.SOL"
SOIL_ID = "SG12387893"
PLANTING_DATE = datetime(2024, 6, 15)

In [6]:

# --- 1. Weather ---
weather_station = WeatherStation.from_files([WEATHER_FILE_PATH])
print("✅ Weather station loaded.")

✅ Weather station loaded.


In [7]:

# --- 2. Soil ---
soil_profile = SoilProfile.from_file(SOIL_ID, SOIL_FILE_PATH)
print("✅ Soil profile loaded.")


✅ Soil profile loaded.


In [8]:

# --- 3. Cultivar ---
cultivar = crop.Sorghum('IB0026')
print("✅ Cultivar set.")

# --- 4. FileX (Management) ---
field = filex.Field(
    id_field='REALTIME', wsta=weather_station, id_soil=soil_profile
)

planting = filex.Planting(
    pdate=PLANTING_DATE, ppop=18, ppoe=18, plme='S',
    plds='R', plrs=45, plrd=0, pldp=5
)

# --- THE FIX IS HERE ---
# Re-introducing an explicit InitialConditions object, but making it dynamic.
# This is the most robust way to ensure consistency.

# 1. Get the layer depths directly from the loaded soil_profile object.
soil_depths = [int(layer['slb']) for layer in soil_profile.table]

# 2. Create a table with high initial water content for each layer.
initial_values_table = []
for depth in soil_depths:
    # (depth, initial water content, initial NH4, initial NO3)
    initial_values_table.append((depth, 0.35, 1.5, 1.5)) 

# 3. Create the InitialConditions object with the dynamic table.
initial_conditions = filex.InitialConditions(
    pcr='SG',
    icdat=PLANTING_DATE,
    icres=1300, 
    icren=0.5,
    table=pd.DataFrame(initial_values_table, columns=['icbl', 'sh2o', 'snh4', 'sno3'])
)
# --- END OF FIX ---


# --- IRRIGATION SECTION ---
irrigation_events = []
for day in range(90):
    irrigation_date = PLANTING_DATE + timedelta(days=day)
    irrigation_events.append(
        filex.IrrigationEvent(idate=irrigation_date, irop='IR001', irval=5)
    )
irrigation = filex.Irrigation(table=irrigation_events)


simulation_controls = filex.SimulationControls(
    general=filex.SCGeneral(sdate=PLANTING_DATE),
    options=filex.SCOptions(water='Y', nitro='N', symbi='N'), # Keep Nitrogen OFF for this test
    methods=filex.SCMethods(infil='S'),
    management=filex.SCManagement(irrig='A', ferti='N', resid='N', harvs='M') # Irrigation is ON
)
print("✅ Management files configured.")

# --- 5. Run Simulation ---
try:
    workspace = os.path.join("dummy")
    os.makedirs(workspace, exist_ok=True)
    
    shutil.copy(WEATHER_FILE_PATH, workspace)
    shutil.copy(SOIL_FILE_PATH, workspace)
    print(f"✔️ Manually copied input files to '{workspace}'")
    
    dssat = DSSAT(workspace)
    
    print(f"🚀 Running simulation in: {workspace}")
    
    results = dssat.run_treatment(
        field=field,
        cultivar=cultivar,
        planting=planting,
        initial_conditions=initial_conditions, # <-- Pass the new object
        irrigation=irrigation,
        simulation_controls=simulation_controls
    )

    print("\n--- SIMULATION COMPLETE ---\n")
    print("Summary Output (Evaluate.OUT):")
    print(results)
    
    if 'PlantGro' in dssat.output_tables:
        print("\nDaily Plant Growth (First 10 days):")
        print(dssat.output_tables['PlantGro'].head(10))

except Exception as e:
    print(f"❌ An error occurred during the DSSAT run: {e}")


✅ Cultivar set.
✅ Management files configured.
✔️ Manually copied input files to 'dummy'
dummy created.
🚀 Running simulation in: dummy

RUN    TRT FLO MAT TOPWT HARWT  RAIN  TIRR   CET  PESW  TNUP  TNLF   TSON TSOC
           dap dap kg/ha kg/ha    mm    mm    mm    mm kg/ha kg/ha  kg/ha t/ha
  1 SG   1 -99 -99     0     0     0     0     2   211     0   -99      0  187

--- SIMULATION COMPLETE ---

Summary Output (Evaluate.OUT):
{'flo': None, 'mat': None, 'topwt': 0, 'harwt': 0, 'rain': 0, 'tirr': 0, 'cet': 2, 'pesw': 211, 'tnup': 0, 'tnlf': None, 'tson': 0, 'tsoc': 187}

Daily Plant Growth (First 10 days):
           @YEAR  DOY  DAS  DAP  L#SD  GSTD  LAID  LWAD  SWAD  GWAD  ...  \
2024-06-15  2024  167    1    0   0.0     0   0.0     0     0     0  ...   

            RL7D  RL8D  RL9D  RL10D  CDAD  LDAD  SDAD  SNW0C  SNW1C  DTTD  
2024-06-15   0.0   0.0   0.0    0.0     0     0     0      0      0  1.85  

[1 rows x 50 columns]
